# Spell Checking via Text Prediction Exploration
## Nicholas Miklaucic & Peabody Work Duty Group

In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from string import punctuation, whitespace
import spellcheck

%matplotlib inline

In [79]:
spellcheck.sentence_correct("A blob is nice this timem of day.")

'A blog in nice his time of day.'

In [81]:
spellcheck.sentence_correct("Name urude quartzite stone.")

'Name crude quartzite stone.'

In [56]:
unwanted_chars = list(punctuation) + list(whitespace)
unwanted_chars.remove(' ')
unwanted_chars.remove('-')
unwanted_chars.remove("'")
def word_parse(string):
    """Lowercases, emoves punctutation besides that which can appear in the inside of words (hyphen, apostrophe), and removes extraneous whitespace."""
    parsed = string.strip().lower()
    for unwanted_char in unwanted_chars:
        parsed = parsed.replace(unwanted_char, '')
    return parsed

In [58]:
other_df = pd.read_csv("test.csv")

fields = ["Locality", "Site", "Name", "Situation"]
other_df = other_df[fields]
other_df[fields] = other_df[fields].apply(np.vectorize(lambda x: word_parse(str(x))))
other_df.head()

,Locality,Site,Name,Situation
0,locality squibnocket head southwest side 0mart...,site squibnocket cliff,name butt of arrowhead,situation on sand under shell just south of st...
1,lmnmw squibnocket head southwest side 0martha'...,site squibnocket cliff,name butt of quartz knife,situation black sandy loam near stake 2
2,locality squibnocket head southwest side ofmar...,site squibnocket cliff,name crude quartz point,situation black sandy loam 1m east of stake a
3,locality squibnocket head southwest side 0mart...,site squibnocket cliff,name urude quartzite spear,situation top of first shell layer
4,uxﬂky squibnocket head southwest side ofmartha...,site squibnocket cliff,name crude chopper,situation underneath lowest shell layer


In [70]:
words = Counter()
for field in fields:
    words.update(Counter(other_df[field].apply(lambda x: x + ' ').sum().strip().split(' ')))
for word, freq in words.most_common():
    words[word.title()] = words[word]
del words['']

In [71]:
words.most_common(10)

[('Site', 2955),
 ('site', 2955),
 ('Locality', 2726),
 ('locality', 2726),
 ('Name', 2672),
 ('name', 2672),
 ('situation', 2592),
 ('Situation', 2592),
 ('shell', 2319),
 ('Shell', 2319)]

The current idea I have for mixing frequencies is as follows: for any word currently in the corpus, ignore it. Otherwise, set it to a constant times the place in the list.

In [74]:
ALPHA = .05 # determines how much normal English words are weighted
# for the value, you should probably use the wanted "top" value / 10000
with open("google-10000-english-no-swears.txt", 'r') as corpusfile:
    for i, word in enumerate(reversed(list(corpusfile))):
        if word.strip() in words:
            continue
        else:
            words[word.strip()] = int(ALPHA * i)
            words[word.strip().title()] = int(ALPHA * i)

In [75]:
words.most_common(50)

[('Site', 2955),
 ('site', 2955),
 ('Locality', 2726),
 ('locality', 2726),
 ('name', 2672),
 ('Name', 2672),
 ('Situation', 2592),
 ('situation', 2592),
 ('shell', 2319),
 ('Shell', 2319),
 ('Heap', 1589),
 ('heap', 1589),
 ('of', 1576),
 ('Of', 1576),
 ('Falls', 1364),
 ('falls', 1364),
 ('nevin', 1339),
 ('Nevin', 1339),
 ('bluehill', 1181),
 ('Bluehill', 1181),
 ('section', 1127),
 ('Section', 1127),
 ('Trench', 1111),
 ('trench', 1111),
 ('Me', 989),
 ('me', 989),
 ('In', 942),
 ('in', 942),
 ('east', 502),
 ('East', 502),
 ('can', 494),
 ('Can', 494),
 ('Home', 494),
 ('will', 494),
 ('More', 494),
 ('home', 494),
 ('your', 494),
 ('Your', 494),
 ('Will', 494),
 ('more', 494),
 ('Search', 493),
 ('time', 493),
 ('Other', 493),
 ('other', 493),
 ('Free', 493),
 ('Time', 493),
 ('search', 493),
 ('Our', 493),
 ('our', 493),
 ('free', 493)]

In [68]:
BETA = 200
with open("name_list.dat", 'r') as namelistfile:
    for line in namelistfile:
        processed = line.strip().lower()
        if processed in words:
            pass
        else:
            words[processed] = BETA
            words[processed.title()] = BETA

In [76]:
del words['i']
with open("spellcheckcorpus.dat", 'w') as outputfile:
    with open("spellcheckcorpuswithfreqs.csv", 'w') as outputfreqs:
        for word, freq in words.most_common():
            outputfile.write(word + '\n')
            outputfreqs.write("{},{}\n".format(word, freq))

In [14]:
# algorithm for computing edit distance
# Good artists copy, great artists steal
# this is from https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
# and I take no credit for it whatsoever
def levenshtein(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]

In [15]:
# testing examples
print(levenshtein("history", "herstory"))
# should output 2: add 'r', change 'i' to 'e'
print(levenshtein("t3sstin", "testing"))
# should output 3: change '3' to 'e', delete 's', add a 'g'

2
3


In [16]:
# now to spell-check a single word, we find its closest thing in the list of words we have and then settle ties by commonality in the list
def correct(input_word):
    """Returns the closest words in the list of words we have, sorted by likelihood."""
    candidates = []
    curr_min_distance = 200
    for word in words:
        distance = levenshtein(word, input_word)
        if distance < curr_min_distance:
            candidates = [word]
            curr_min_distance = distance
        elif distance == curr_min_distance:
            candidates.append(word)
        else:
            continue
    candidates.sort(key=lambda x: words[x], reverse=True)
    return candidates

In [17]:
print(correct("arrop oint"))
print(correct("or1g1nal3fdf"))
print(correct("history"))
print(correct("mcmurphy"))

['arrowpoint']
['originally', 'original']
['history']
['murphy']


I feel really good about this system, especially if I ever get the actual English dictionary to back the wordlist up (EDIT: done!) and figure out how to properly mix those. Things to think about improving:
 * If the OCR collapses one word into many or many words into one, that's really hard for this to catch.
 * I thought about doing totally next-level optical edit distance stuff, but that seems like overkill.
 * Knowing when to apply this system is also important: locations and names have to stay that way, and Massachusetts has some *weird* place names that can't be spell-checked and might change constantly.
 * Word embeddings a la `word2vec` would significantly improve this and I'm working on that.

The really nifty thing would be to use this algorithm to train a neural network to do deep learning on spell-checking (which has been done successfully), but that REALLY seems like overkill. I'll plug this into the whole pipeline and get the dictionary sorted before I do that.